# Demonstation of raytracing in Kotlin

In [12]:
%use lets-plot

In [13]:
class Point(
    var x: Float = 0f,
    var y: Float = 0f,
    var z: Float = 0f
) {
    fun getPointOnScreen(cameraPoint: Point): Pair<Float, Float> {

        if (z <= 0) {
            throw IllegalArgumentException("The point is behind the screen")
        }

        // The focal length is the distance between the camera and the screen.
        val focalLength = cameraPoint.z * -1

        // The distance between the point and the camera.
        val zDistance = z - cameraPoint.z

        // the x and y coordinates of the point on the screen.
        val xProjected = (x * focalLength) / (focalLength + zDistance)

        val yProjected = (y * focalLength) / (focalLength + zDistance)

        return Pair(xProjected, yProjected)
    }
}

// Defines the point of the camera in 3D space.
// The camera has to be in front if the screen whitch means that the z coordinate has to be negative.
val camera = Point(0f, 0f, -1f)

// Defines all the points of a square in 3D space.
val square: List<Point> = listOf(
    Point(2f, -2f, 5f),
    Point(2f, 2f, 5f),
    Point(-2f, 2f, 5f),
    Point(-2f, -2f, 5f),
    Point(2f, -2f, 7f),
    Point(2f, 2f, 7f),
    Point(-2f, 2f, 7f),
    Point(-2f, -2f, 7f),
)

// Get the points coordinates on the screen.
val pointsOnScreen = square.map { it.getPointOnScreen(camera) }


// Display the data on a plot
val data = mapOf("x" to pointsOnScreen.map { it.first }, "y" to pointsOnScreen.map { it.second })
val plot = letsPlot(data) { x = "x"; y = "y" }
val points = geomPoint()
plot + points


